In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

使用自定义容器在CPU上使用Vertex AI培训的PyTorch图像分类多节点分布式数据并行训练

# 在谷歌协作平台中查看
# 在谷歌云协作企业版中查看
# 在工作台中打开
# 在GitHub上查看

请点击链接以获取更多信息。

## 概览

本教程演示了如何使用 Vertex AI SDK for Python 和自定义容器创建一个分布式 PyTorch 训练作业。这将帮助您的训练作业扩展以处理大量数据。

了解更多关于[Vertex AI Training](https://cloud.google.com/vertex-ai/docs/training/custom-training)。

### 目标

在本教程中，您将学习如何使用Vertex AI SDK和自定义容器创建一个分布式的PyTorch训练作业。您将设置GCP来使用自定义容器、Vertex TensorBoard实例并运行自定义训练作业。

本教程使用以下Google Cloud ML服务：

- `Vertex AI SDK`
- `Vertex AI TensorBoard`
- `CustomContainerTrainingJob`
- `Artifact Registry`

执行的步骤包括：

- 设置您的GCP项目：设置PROJECT_ID、LOCATION及SERVICE_ACCOUNT
- 创建一个云存储桶
- 使用Artifact Registry和Docker构建自定义容器
- 创建一个Vertex AI TensorBoard实例来存储您的Vertex AI实验
- 运行一个Vertex AI SDK CustomContainerTrainingJob

### 数据集

本教程使用的数据集是<a href="http://yann.lecun.com/exdb/mnist/">MNIST数据库</a>。手写数字MNIST数据库有一个包含60,000个样本的训练集和一个包含10,000个样本的测试集。它是来自NIST的一个更大数据集的子集。这些数字经过了尺寸规范化，并居中在固定大小的图像中。

成本

本教程使用谷歌云的计费组件：
* Vertex AI
* 云存储
* Vertex AI TensorBoard

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[云存储定价](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)，根据您预期的使用量生成成本估算。

开始吧

### 为Python安装Vertex AI SDK和其他必需的软件包

In [ ]:
! pip3 install --upgrade google-cloud-aiplatform -q

重新启动运行时（仅限Colab）

为了使用新安装的软件包，您必须重新启动Google Colab上的运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning"> <b>⚠️ 内核将重新启动。在继续下一步之前，请等待它完成。⚠️</b> </div>

### 在谷歌Colab上验证您的笔记本环境

在谷歌Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置Google Cloud项目信息并初始化用于Python的Vertex AI SDK

要开始使用Vertex AI，您必须拥有一个现有的Google Cloud项目并[启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = "gs://your-bucket-name-unique"  # @param {type:"string"}

只有在您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $LOCATION -p $PROJECT_ID $BUCKET_URI

### 导入库

In [ ]:
from google.cloud import aiplatform

### 初始化用于 Python 的 Vertex AI SDK

为您的项目初始化用于 Python 的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

#### 服务账号

您使用服务账号运行Vetex AI CustomContainerTrainingJob。如果您不想使用您的项目的Compute Engine服务账号，请将 `SERVICE_ACCOUNT` 设定为另一个服务账号ID。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

### 创建自定义的Python训练包

在你可以进行本地训练之前，你必须创建源代码文件、要求文件和Docker文件。

你将创建一个目录，并将所有文件写入该文件夹中。

In [ ]:
PYTHON_PACKAGE_APPLICATION_DIR = "trainer"

In [ ]:
!mkdir -p $PYTHON_PACKAGE_APPLICATION_DIR

### 编写培训脚本

In [ ]:
%%writefile {PYTHON_PACKAGE_APPLICATION_DIR}/task.py


"""
Main program for PyTorch distributed training.
Adapted from: https://github.com/narumiruna/pytorch-distributed-example
"""

import argparse
import os
import shutil

import torch
from torch import distributed
from torch.nn.parallel import DistributedDataParallel
from torch.utils import data
from torch.utils.tensorboard import SummaryWriter

from torchvision import datasets, transforms

def parse_args():

  parser = argparse.ArgumentParser()

  # Using environment variables for Cloud Storage directories
  # see more details in https://cloud.google.com/vertex-ai/docs/training/code-requirements
  parser.add_argument(
      '--model-dir', default=os.getenv('AIP_MODEL_DIR'), type=str,
      help='a Cloud Storage URI of a directory intended for saving model artifacts')
  parser.add_argument(
      '--tensorboard-log-dir', default=os.getenv('AIP_TENSORBOARD_LOG_DIR'), type=str,
      help='a Cloud Storage URI of a directory intended for saving TensorBoard')
  parser.add_argument(
      '--checkpoint-dir', default=os.getenv('AIP_CHECKPOINT_DIR'), type=str,
      help='a Cloud Storage URI of a directory intended for saving checkpoints')

  parser.add_argument(
      '--backend', type=str, default='gloo',
      help='Use the `nccl` backend for distributed GPU training.'
           'Use the `gloo` backend for distributed CPU training.')
  parser.add_argument(
      '--init-method', type=str, default='env://',
      help='URL specifying how to initialize the package.')
  parser.add_argument(
      '--world-size', type=int, default=os.environ.get('WORLD_SIZE', 1),
      help='The total number of nodes in the cluster. '
           'This variable has the same value on every node.')
  parser.add_argument(
      '--rank', type=int, default=os.environ.get('RANK', 0),
      help='A unique identifier for each node. '
           'On the master worker, this is set to 0. '
           'On each worker, it is set to a different value from 1 to WORLD_SIZE - 1.')
  parser.add_argument(
      '--epochs', type=int, default=20)
  parser.add_argument(
      '--no-cuda', action='store_true')
  parser.add_argument(
      '-lr', '--learning-rate', type=float, default=1e-3)
  parser.add_argument(
      '--batch-size', type=int, default=128)
  parser.add_argument(
      '--local-mode', action='store_true', help='use local mode when running on your local machine')

  args = parser.parse_args()

  return args

def makedirs(model_dir):
  if os.path.exists(model_dir) and os.path.isdir(model_dir):
    shutil.rmtree(model_dir)
  os.makedirs(model_dir)
  return

def distributed_is_initialized():
  if distributed.is_available():
    if distributed.is_initialized():
      return True
  return False

class Average(object):

  def __init__(self):
    self.sum = 0
    self.count = 0

  def __str__(self):
    return '{:.6f}'.format(self.average)

  @property
  def average(self):
    return self.sum / self.count

  def update(self, value, number):
    self.sum += value * number
    self.count += number

class Accuracy(object):

  def __init__(self):
    self.correct = 0
    self.count = 0

  def __str__(self):
    return '{:.2f}%'.format(self.accuracy * 100)

  @property
  def accuracy(self):
    return self.correct / self.count

  @torch.no_grad()
  def update(self, output, target):
    pred = output.argmax(dim=1)
    correct = pred.eq(target).sum().item()

    self.correct += correct
    self.count += output.size(0)

class Net(torch.nn.Module):

  def __init__(self, device):
    super(Net, self).__init__()
    self.fc = torch.nn.Linear(784, 10).to(device)

  def forward(self, x):
    return self.fc(x.view(x.size(0), -1))

class MNISTDataLoader(data.DataLoader):

  def __init__(self, root, batch_size, train=True):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)),
    ])

    dataset = datasets.MNIST(root, train=train, transform=transform, download=True)
    sampler = None
    if train and distributed_is_initialized():
      sampler = data.DistributedSampler(dataset)

    super(MNISTDataLoader, self).__init__(
        dataset,
        batch_size=batch_size,
        shuffle=(sampler is None),
        sampler=sampler,
    )

class Trainer(object):

  def __init__(self,
      model,
      optimizer,
      train_loader,
      test_loader,
      device,
      model_name,
      checkpoint_path
  ):
    self.model = model
    self.optimizer = optimizer
    self.train_loader = train_loader
    self.test_loader = test_loader
    self.device = device
    self.model_name = model_name
    self.checkpoint_path = checkpoint_path

  def save(self, model_dir):
    model_path = os.path.join(model_dir, self.model_name)
    torch.save(self.model.state_dict(), model_path)

  def fit(self, epochs, is_chief, writer):

    for epoch in range(1, epochs + 1):

      print('Epoch: {}, Training ...'.format(epoch))
      train_loss, train_acc = self.train()

      if is_chief:
        test_loss, test_acc = self.evaluate()
        writer.add_scalar('Loss/train', train_loss.average, epoch)
        writer.add_scalar('Loss/test', test_loss.average, epoch)
        writer.add_scalar('Accuracy/train', train_acc.accuracy, epoch)
        writer.add_scalar('Accuracy/test', test_acc.accuracy, epoch)
        torch.save(self.model.state_dict(), self.checkpoint_path)

        print(
            'Epoch: {}/{},'.format(epoch, epochs),
            'train loss: {}, train acc: {},'.format(train_loss, train_acc),
            'test loss: {}, test acc: {}.'.format(test_loss, test_acc),
        )

  def train(self):

    self.model.train()

    train_loss = Average()
    train_acc = Accuracy()

    for data, target in self.train_loader:
      data = data.to(self.device)
      target = target.to(self.device)

      output = self.model(data)
      loss = torch.nn.functional.cross_entropy(output, target)

      self.optimizer.zero_grad()
      loss.backward()
      self.optimizer.step()

      train_loss.update(loss.item(), data.size(0))
      train_acc.update(output, target)

    return train_loss, train_acc

  @torch.no_grad()
  def evaluate(self):
    self.model.eval()

    test_loss = Average()
    test_acc = Accuracy()

    for data, target in self.test_loader:
      data = data.to(self.device)
      target = target.to(self.device)

      output = self.model(data)
      loss = torch.nn.functional.cross_entropy(output, target)

      test_loss.update(loss.item(), data.size(0))
      test_acc.update(output, target)

    return test_loss, test_acc

def main():

  args = parse_args()

  local_data_dir = './tmp/data'
  local_model_dir = './tmp/model'
  local_tensorboard_log_dir = './tmp/logs'
  local_checkpoint_dir = './tmp/checkpoints'

  model_dir = args.model_dir or local_model_dir
  tensorboard_log_dir = args.tensorboard_log_dir or local_tensorboard_log_dir
  checkpoint_dir = args.checkpoint_dir or local_checkpoint_dir

  gs_prefix = 'gs://'
  gcsfuse_prefix = '/gcs/'
  if model_dir and model_dir.startswith(gs_prefix):
    model_dir = model_dir.replace(gs_prefix, gcsfuse_prefix)
  if tensorboard_log_dir and tensorboard_log_dir.startswith(gs_prefix):
    tensorboard_log_dir = tensorboard_log_dir.replace(gs_prefix, gcsfuse_prefix)
  if checkpoint_dir and checkpoint_dir.startswith(gs_prefix):
    checkpoint_dir = checkpoint_dir.replace(gs_prefix, gcsfuse_prefix)

  writer = SummaryWriter(tensorboard_log_dir)

  is_chief = args.rank == 0
  if is_chief:
    makedirs(checkpoint_dir)
    print(f'Checkpoints will be saved to {checkpoint_dir}')

  checkpoint_path = os.path.join(checkpoint_dir, 'checkpoint.pt')
  print(f'checkpoint_path is {checkpoint_path}')

  if args.world_size > 1:
    print('Initializing distributed backend with {} nodes'.format(args.world_size))
    distributed.init_process_group(
          backend=args.backend,
          init_method=args.init_method,
          world_size=args.world_size,
          rank=args.rank,
      )
    print(f'[{os.getpid()}]: '
          f'world_size = {distributed.get_world_size()}, '
          f'rank = {distributed.get_rank()}, '
          f'backend={distributed.get_backend()} \n', end='')

  if torch.cuda.is_available() and not args.no_cuda:
    device = torch.device('cuda:{}'.format(args.rank))
  else:
    device = torch.device('cpu')

  model = Net(device=device)
  if distributed_is_initialized():
    model.to(device)
    model = DistributedDataParallel(model)

  if is_chief:
    # All processes should see same parameters as they all start from same
    # random parameters and gradients are synchronized in backward passes.
    # Therefore, saving it in one process is sufficient.
    torch.save(model.state_dict(), checkpoint_path)
    print(f'Initial chief checkpoint is saved to {checkpoint_path}')

  # Use a barrier() to make sure that process 1 loads the model after process
  # 0 saves it.
  if distributed_is_initialized():
    distributed.barrier()
    # configure map_location properly
    model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    print(f'Initial chief checkpoint is saved to {checkpoint_path} with map_location {device}')
  else:
    model.load_state_dict(torch.load(checkpoint_path))
    print(f'Initial chief checkpoint is loaded from {checkpoint_path}')

  optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)

  train_loader = MNISTDataLoader(
      local_data_dir, args.batch_size, train=True)
  test_loader = MNISTDataLoader(
      local_data_dir, args.batch_size, train=False)

  trainer = Trainer(
      model=model,
      optimizer=optimizer,
      train_loader=train_loader,
      test_loader=test_loader,
      device=device,
      model_name='mnist.pt',
      checkpoint_path=checkpoint_path,
  )
  trainer.fit(args.epochs, is_chief, writer)

  if model_dir == local_model_dir:
    makedirs(model_dir)
    trainer.save(model_dir)
    print(f'Model is saved to {model_dir}')

  print(f'Tensorboard logs are saved to: {tensorboard_log_dir}')

  writer.close()

  if is_chief:
    os.remove(checkpoint_path)

  if distributed_is_initialized():
    distributed.destroy_process_group()

  return

if __name__ == '__main__':
  main()


### 写需求文件

In [ ]:
%%writefile {PYTHON_PACKAGE_APPLICATION_DIR}/requirements.txt

torch
torchvision
tensorboard

写docker文件

In [ ]:
%%writefile {PYTHON_PACKAGE_APPLICATION_DIR}/Dockerfile


FROM pytorch/pytorch:1.8.1-cuda11.1-cudnn8-runtime

RUN apt-get update && \
    apt-get install -y curl gnupg && \
    echo "deb [signed-by=/usr/share/keyrings/cloud.google.gpg] http://packages.cloud.google.com/apt cloud-sdk main" | tee -a /etc/apt/sources.list.d/google-cloud-sdk.list && \
    curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key --keyring /usr/share/keyrings/cloud.google.gpg  add - && \
    apt-get update -y && \
    apt-get install google-cloud-sdk -y

COPY . /trainer

WORKDIR /trainer

RUN pip install -r requirements.txt

ENTRYPOINT ["python", "-m", "task"]


本地培训

In [ ]:
! ls trainer
! cat trainer/requirements.txt
! pip install -r trainer/requirements.txt
! cat trainer/task.py

In [ ]:
%run trainer/task.py --epochs 5 --no-cuda --local-mode

In [ ]:
! ls ./tmp

清理临时文件

In [ ]:
! rm -rf ./tmp

### 使用自定义容器进行Vertex AI训练

### 构建自定义容器

启用Artifact Registry API
您必须为您的项目启用Artifact Registry API服务。

<a href="https://cloud.google.com/artifact-registry/docs/enable-service">了解有关启用服务的更多信息</a>。

In [ ]:
! gcloud services enable artifactregistry.googleapis.com

if os.getenv("IS_TESTING"):
    ! sudo apt-get update --yes && sudo apt-get --only-upgrade --yes install google-cloud-sdk-cloud-run-proxy google-cloud-sdk-harbourbridge google-cloud-sdk-cbt google-cloud-sdk-gke-gcloud-auth-plugin google-cloud-sdk-kpt google-cloud-sdk-local-extract google-cloud-sdk-minikube google-cloud-sdk-app-engine-java google-cloud-sdk-app-engine-go google-cloud-sdk-app-engine-python google-cloud-sdk-spanner-emulator google-cloud-sdk-bigtable-emulator google-cloud-sdk-nomos google-cloud-sdk-package-go-module google-cloud-sdk-firestore-emulator kubectl google-cloud-sdk-datastore-emulator google-cloud-sdk-app-engine-python-extras google-cloud-sdk-cloud-build-local google-cloud-sdk-kubectl-oidc google-cloud-sdk-anthos-auth google-cloud-sdk-app-engine-grpc google-cloud-sdk-pubsub-emulator google-cloud-sdk-datalab google-cloud-sdk-skaffold google-cloud-sdk google-cloud-sdk-terraform-tools google-cloud-sdk-config-connector
    ! gcloud components update --quiet

### 创建一个私有的 Docker 仓库
您的第一步是在 Google Artifact Registry 中创建自己的 Docker 仓库。

1 - 运行 gcloud artifacts repositories create 命令，在您的区域创建一个新的 Docker 仓库，并使用描述“docker 仓库”。

2 - 运行 gcloud artifacts repositories list 命令来验证您的仓库是否已创建。

In [ ]:
PRIVATE_REPO = "my-docker-repo"

! gcloud artifacts repositories create {PRIVATE_REPO} --repository-format=docker --location={LOCATION} --description="Docker repository"

! gcloud artifacts repositories list

In [ ]:
DEPLOY_IMAGE = (
    f"{LOCATION}-docker.pkg.dev/" + PROJECT_ID + f"/{PRIVATE_REPO}" + "/tf_serving"
)

In [ ]:
print("Deployment:", DEPLOY_IMAGE)

# 在工作台执行

### 配置私有存储库的身份验证
在推送或拉取容器镜像之前，请配置Docker使用gcloud命令行工具对您地区的Artifact Registry发出的请求进行身份验证。

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules

if not IS_COLAB:
    ! gcloud auth configure-docker {LOCATION}-docker.pkg.dev --quiet

### Docker容器镜像用于提供
设定用于提供预测的TensorFlow Serving Docker容器镜像。

1. 从Docker Hub上拉取相对应的CPU或GPU TensorFlow Serving Docker镜像。
2. 为镜像创建一个标签以在Artifact Registry注册。
3. 将镜像注册到Artifact Registry。

<a href="https://www.tensorflow.org/tfx/serving/docker">了解更多关于TensorFlow Serving的信息</a>。

In [ ]:
if not IS_COLAB:
    ! cd trainer && docker build -t $DEPLOY_IMAGE -f Dockerfile .
    ! docker run --rm $DEPLOY_IMAGE --epochs 5 --no-cuda --local-mode
    ! docker push $DEPLOY_IMAGE

在Colab中执行

使用Cloud Build构建和推送一个Docker镜像

In [ ]:
if IS_COLAB:
    ! cd trainer && gcloud builds submit --timeout=1800s --region={LOCATION} --tag $DEPLOY_IMAGE

### 创建一个 Vertex AI TensorBoard 实例

注意: <a href="https://cloud.google.com/vertex-ai/pricing#tensorboard">Vertex AI TensorBoard </a> 每个唯一活跃用户收取每月 $300 的费用。活跃用户通过 Vertex AI TensorBoard UI 进行衡量。您还需为使用 Vertex AI TensorBoard 的 Google Cloud 资源付费，例如存储在 Cloud Storage 中的 TensorBoard 日志。</a> 请查看上面的链接以获取最新价格信息。

In [ ]:
content_name = "pt-img-cls-multi-node-ddp-cust-cont"
content_name = content_name + "-cpu-unique"

In [ ]:
tensorboard = aiplatform.Tensorboard.create(
    display_name=content_name,
)

选项：使用之前创建的Vertex AI Tensorboard实例

```
tensorboard_name =“您的Tensorboard资源名称或Tensorboard ID”
tensorboard = aiplatform.Tensorboard（tensorboard_name = tensorboard_name）
```

运行Vertex AI SDK CustomContainerTrainingJob

In [ ]:
display_name = content_name
gcs_output_uri_prefix = f"{BUCKET_URI}/{display_name}"

replica_count = 4
machine_type = "n1-standard-4"

args = [
    "--backend",
    "gloo",
    "--no-cuda",
    "--batch-size",
    "128",
    "--epochs",
    "25",
]

In [ ]:
custom_container_training_job = aiplatform.CustomContainerTrainingJob(
    display_name=display_name,
    container_uri=DEPLOY_IMAGE,
)

In [ ]:
custom_container_training_job.run(
    args=args,
    base_output_dir=gcs_output_uri_prefix,
    replica_count=replica_count,
    machine_type=machine_type,
    tensorboard=tensorboard.resource_name,
    service_account=SERVICE_ACCOUNT,
)

In [ ]:
print(f"Custom Training Job Name: {custom_container_training_job.resource_name}")
print(f"GCS Output URI Prefix: {gcs_output_uri_prefix}")

### 查看培训输出成果

In [ ]:
! gsutil ls $gcs_output_uri_prefix

清理

要清理此项目中使用的所有 Google Cloud 资源，您可以[删除用于此教程的 Google Cloud 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除此教程中创建的各个资源：

- Vertex AI Tensorboard
- 云存储桶
- 工件存储库
- 生成的文件/文件夹

In [ ]:
# Set this to true only if you'd like to delete your bucket
delete_bucket = False

# Set this to true only if you'd like to delete your tensorboard
delete_tensorboard = False

# Set this to true only if you'd like to delete your artifact repository
delete_artifact_repository = False

! gsutil rm -rf $gcs_output_uri_prefix

! rm -rf ./trainer

if delete_artifact_repository:
    !gcloud artifacts repositories delete {PRIVATE_REPO} --location={LOCATION} --quiet

if delete_bucket:
    ! gsutil rm -r $BUCKET_URI

if delete_tensorboard:
    tensorboard.delete()